In [15]:
import sys
import os
sys.path.append('../utils/analysis/')

import pandas as pd
import polars as pl
from filtering import Filter

In [16]:
file = "../physionet.org/files/mimiciv/3.1/icu/chartevents.csv.gz"
run_tests = False

In [17]:
filter = Filter(file, "chartevents", debug=True)
if run_tests:
    df = filter.search_subject(10000690)

[Filter] Initialized for chartevents with 313645063 rows, sorted by subject_id


In [18]:
subject_files = '../data/icu_unique_subject_ids.csv'

if not os.path.exists(subject_files):
    # Read the CSV file and get unique subject_ids
    unique_subject_ids = pd.read_csv(file, usecols=['subject_id'])['subject_id'].unique()

    # Convert to DataFrame and save to CSV
    unique_df = pd.DataFrame(unique_subject_ids, columns=['subject_id'])
    unique_df.to_csv(subject_files, index=False)
else:
    print("Subject ID file already exists.")

Subject ID file already exists.


In [19]:
# Load subject_files into df
df_subjects = pd.read_csv(subject_files)

# Get Random Patient and print
random_patient_id = df_subjects.sample(n=1)['subject_id'].values[0]
print(f"Random Patient ID: {random_patient_id}")

Random Patient ID: 10370700


In [ ]:
df = filter.search_subject(random_patient_id)